In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 8
Maxiterations_cotel = 5
Maxiterations_turbine = 15

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 4
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 10

n_GTU_it = [0]
if Сalculate_minimum == True:
    gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        n_GTU_it.append(round(n_GTU, 5))
        print("n_GTU:", n_GTU_it)
        if i < 4:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_n_GTU = abs((n_GTU_it[-1] - n_GTU_it[-2]) / n_GTU_it[-1] * 100)
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd, Delta_n_GTU)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 3
        GTU_input.at["n", 1] = n_GTU
        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        gas_streams = Calculate_CCGT_PKM_iter(
            arguments_all_it, New_Iter_pkm, pkm_pgu_tol
        )
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")

        if abs(Delta_min) < Calctolerance:
            arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )
            n_GTU_it.append(round(n_GTU, 5))
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, New_Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            print("n_GTU_it", n_GTU_it)
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )
            print("n_GTU_it", n_GTU_it)

n_GTU: [0, 1]
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 48.8
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  13.5 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Время 2 итерации расчета КУ+ТУ:---  23.9 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.60228, 0.46265, 0.44032]
Pvd_it [7.45517, 5.00265, 3.95103]
fin КУ и ТУ:--- 23.9 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 27.1 сек. ---
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 115.7
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  10.7 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Давление на входе в ЦНД меньше нуля при температуре 

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,527.600273,0.1,941.958209,386.887312,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.689692,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,563.534977,0.1,1012.578258,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
PEVD-IVD,484.609704,0.1,921.136418,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
IVD-EVD,266.234512,0.1,676.843790,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
EVD-PPND,203.562262,0.1,608.967473,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
PPND-IND,201.192222,0.1,606.417599,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
IND-GPK,149.562221,0.1,551.144838,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762
GPK-out,98.468322,0.1,496.900890,439.577003,0.771734,0.109778,0.037498,0.074227,0.006762


In [3]:
water_streams

,T,P,H,G,S,X
AIR,-29.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,497.825628,3.762877,3443.762321,61.709690,NaN,NaN
IVD-PEVD,256.576841,4.436881,2798.360963,61.709690,NaN,NaN
EVD-IVD,247.838861,4.436881,1075.269660,61.709690,NaN,NaN
PEN-EVD,141.096437,4.436881,596.493909,61.709690,NaN,NaN
BND-PEN,139.922385,0.360746,588.828792,61.709690,NaN,NaN
PPND-DROSND,185.359530,0.386296,2832.609468,11.372308,NaN,NaN
IND-PPND,139.922385,0.360746,2733.342540,11.372308,NaN,NaN
GPK-IND,139.922385,0.360746,593.393747,73.081998,NaN,NaN
GPK-REC,139.922385,0.360746,593.393747,72.888384,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,45.441864,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,45.441864,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,45.441864,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,45.441864,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,18.074770,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.473012,0.611858,0.773075,0.573579
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,102.655688,NaN,29.564768
DK,NaN,6.167700,NaN,NaN
Turbine,17.590248,NaN,NaN,NaN
Tots1,10.049399,NaN,NaN,0.831200
Tots2,1.861302,NaN,NaN,0.812464
Tots3,0.466022,NaN,NaN,0.754572
Tots4,5.213524,NaN,NaN,0.574266


In [6]:
heaters

,Qw,Qg,KPD
PEVD,39793.772551,40195.729849,0.99
IVD,106311.567281,107385.421496,0.99
EVD,29538.499215,29836.867894,0.99
PPND,1109.657339,1120.865999,0.99
IND,24053.668607,24296.634957,0.99
GPK,23605.948011,23844.391930,0.99
SP2,1.538265,NaN,0.99
SP1,22.864660,NaN,0.99
OD,1.701481,NaN,0.99
Strans,129819.828989,NaN,NaN


In [7]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0


In [ ]:
PKM_zaryad = False
PKM_razryad = True

Maxiterations_KU_TU = 20
Maxiterations_cotel = 5
Maxiterations_turbine = 20

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm = 20
pkm_pgu_tol = 10**-2
CCGT = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Рассчитана пиковая ГТУ, расход воздуха: 932.7593619719848


In [ ]:
gas_streams

In [ ]:
water_streams

In [ ]:
syngas_streams

In [ ]:
accumulation